Eyüphan Seymen 21040101007 Bilgisayar Mühendisliği

In [33]:
import pandas as pd


In [108]:
df = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv")
converted_data = df.values.tolist()  
head_labels = df.columns.tolist()

df

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea


In [109]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [110]:
unique_vals(converted_data, -5)

{'Normal', 'Obese', 'Overweight'}

In [111]:
def class_counts(rows):
   
    counts = {} 
    for row in rows:
        label = row[-5]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [112]:
class_counts(converted_data)

{'Overweight': 148, 'Normal': 216, 'Obese': 10}

In [113]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [114]:
is_numeric(7)


True

In [134]:
class Question:
  

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
       
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
       
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            head_labels[self.column], condition, str(self.value))

In [116]:
Question(1, "Male")


Is Gender == Male?

In [136]:
def partition(rows, question):
   
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [135]:
true_rows,false_rows=partition(converted_data,Question(1,"Male"))

true_rows

[[1,
  'Male',
  27,
  'Software Engineer',
  6.1,
  6,
  42,
  6,
  'Overweight',
  '126/83',
  77,
  4200,
  nan],
 [2, 'Male', 28, 'Doctor', 6.2, 6, 60, 8, 'Normal', '125/80', 75, 10000, nan],
 [3, 'Male', 28, 'Doctor', 6.2, 6, 60, 8, 'Normal', '125/80', 75, 10000, nan],
 [4,
  'Male',
  28,
  'Sales Representative',
  5.9,
  4,
  30,
  8,
  'Obese',
  '140/90',
  85,
  3000,
  'Sleep Apnea'],
 [5,
  'Male',
  28,
  'Sales Representative',
  5.9,
  4,
  30,
  8,
  'Obese',
  '140/90',
  85,
  3000,
  'Sleep Apnea'],
 [6,
  'Male',
  28,
  'Software Engineer',
  5.9,
  4,
  30,
  8,
  'Obese',
  '140/90',
  85,
  3000,
  'Insomnia'],
 [7,
  'Male',
  29,
  'Teacher',
  6.3,
  6,
  40,
  7,
  'Obese',
  '140/90',
  82,
  3500,
  'Insomnia'],
 [8, 'Male', 29, 'Doctor', 7.8, 7, 75, 6, 'Normal', '120/80', 70, 8000, nan],
 [9, 'Male', 29, 'Doctor', 7.8, 7, 75, 6, 'Normal', '120/80', 70, 8000, nan],
 [10, 'Male', 29, 'Doctor', 7.8, 7, 75, 6, 'Normal', '120/80', 70, 8000, nan],
 [11, 'Male'

In [138]:

class Decision_Node:
    

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [121]:
def gini(rows):
 # impurity hesaplama
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [128]:
def info_gain(left, right, current_uncertainty):
    # kazanilan info hesaplama
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [129]:
def find_best_split(rows):
   
    best_gain = 0  
    best_question = None  
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  

    for col in range(n_features): 

        values = set([row[col] for row in rows]) 

        for val in values:  

            question = Question(col, val)

            
            true_rows, false_rows = partition(rows, question)

           
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

           
            gain = info_gain(true_rows, false_rows, current_uncertainty)

           
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [123]:
class Leaf:
   
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [132]:
def build_tree(rows):
   

    
    gain, question = find_best_split(rows)

   
    if gain == 0:
        return Leaf(rows)

   
    true_rows, false_rows = partition(rows, question)

    
    true_branch = build_tree(true_rows)

   
    false_branch = build_tree(false_rows)

   
    return Decision_Node(question, true_branch, false_branch)

In [131]:
def print_tree(node, spacing=""):
    

  
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

   
    print (spacing + str(node.question))

   
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

 
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [137]:
my_tree = build_tree(converted_data)
print_tree(my_tree)

Is BMI Category == Normal?
--> True:
  Predict {'Normal': 216}
--> False:
  Is Daily Steps >= 4000?
  --> True:
    Predict {'Overweight': 148}
  --> False:
    Predict {'Obese': 10}
